In [1]:
import os
import pydicom
import glob
import pandas as pd
from datetime import datetime


In [2]:
# load segmented cases  
prostate_lesion_seg = pd.read_csv("/data/oleksii/Prostate-Lesion-Datasets-NRRDS/lesion_dataset_1909_20240131_OB.csv", sep=";")

# load cases proposed by Lucas
cases_to_annotate = pd.read_excel("meta_data_lesion/20240702_Lucas_Cases_mit_KI_bewerten.xlsx")

# check for overlap
prostate_lesion_seg[prostate_lesion_seg["study_orthanc_id"].isin(cases_to_annotate["study_orthanc_id"])]

,study_orthanc_id,class,region,zone,segment_id,count,study_date,patient_id,csPCa,case_class,healthy,StudyInstanceUID,class_id


In [5]:
seq_map = pd.read_csv("/home/oleksii/projects/ohif-orthanc-postgres-docker/sequence_mapping/sequence_mapping_13024_studies_20240115.csv", sep=";") 
seq_map.head(3)

,t2w_tra_id,t2w_tra_sd,t2w_sag_id,t2w_sag_sd,t2w_cor_id,t2w_cor_sd,adc_tra_id,adc_tra_sd,adc_sag_id,adc_sag_sd,...,dwi_tra_id,dwi_tra_sd,dwi_sag_id,dwi_sag_sd,dwi_cor_id,dwi_cor_sd,Manufacturer,study_orthanc_id,PatientID,StudyDate
0,7f6c6664-416331da-175b9083-1a67fdfa-064bf673,t2_tse_tra,7cd0bca9-73752b20-fb4f318f-bf79dc39-95166304,t2_tse_sag_p2,10f846e9-f30ceaf6-826e5959-1af9b396-9dd682cc,t2_tse_cor_p2_320,232a75a0-af68dbfe-cbc0afec-f9aa3347-bd6541f7,ep2d_diff_b50_500_1000_tra_adc_dfc,NaN,NaN,...,1ac49251-c9662c57-d70bf60b-0e1db131-19ad825c,ep2d_diff_b50_500_1000_tra_tracew_dfc,NaN,NaN,fcac7d02-2ef30e51-0572b869-91e8f18b-938eb178,ep2d_diff_b50_500_1000_cor_tracew_dfc,siemens,786714b5-a6309313-214532e5-e92d1dff-308695f6,26586,20220201.0
1,b568bc9b-6a9d9e07-65793909-7a453d4c-784a5238,t2_tse_tra,d505ddb2-ea663427-5d079838-795bfb31-4d1f6a02,t2_tse_sag_p2,c036a794-9e549195-c6a59f77-ba67c619-2a70bba1,t2_tse_cor_p2_320,84669307-e90afc45-d3d6f7d0-fef92916-36c603bf,ep2d_diff_b50_500_1000_tra_adc_dfc,NaN,NaN,...,d7dfb776-f693fbfd-62795a90-62e28fda-d1a86544,ep2d_diff_b50_500_1000_tra_tracew_dfc,NaN,NaN,NaN,NaN,siemens,e27d17d9-47c79ade-2218d504-e26738cd-568156b3,31906,20220201.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,siemens,21915507-0546865f-dfde6bc5-77369bc3-bb431e29,31906,20220201.0


In [6]:
# filter non-existing cases out
cases_to_annotate = cases_to_annotate[cases_to_annotate["study_orthanc_id"].isin(seq_map["study_orthanc_id"])]

dataset_annotate = seq_map[seq_map["study_orthanc_id"].isin(cases_to_annotate["study_orthanc_id"])]
dataset_annotate_full = dataset_annotate[~(dataset_annotate['t2w_tra_id'].isna() | dataset_annotate['adc_tra_id'].isna() | dataset_annotate['dwi_tra_id'].isna())]

In [6]:
# remove revently segmented cases from the list
lesion_studies_processed = os.listdir("/data/oleksii/alta-ai.com/alta-ai-orthanc-backup/2024-06-30/prostate_lesion/ProcessingState.PROCESSED")
dataset_annotate_full = dataset_annotate_full[~dataset_annotate_full["study_orthanc_id"].isin(lesion_studies_processed)]

In [8]:
# sent to Lucas for double checking
# dataset_annotate_incomplete_seq = dataset_annotate[~dataset_annotate["study_orthanc_id"].isin(dataset_annotate_full["study_orthanc_id"])]
# dataset_annotate_incomplete_seq.to_csv("annotate_incomplete_seq_map.csv", sep=";")


In [9]:
class_list = pd.read_csv("/home/oleksii/projects/ohif-orthanc-postgres-docker/datasets/classification/klassenliste_matched_with_pacs_20240411.csv", sep=";")
dataset_annotate_class = dataset_annotate_full[~dataset_annotate_full["study_orthanc_id"].isin(class_list["study_orthanc_id"])]
dataset_annotate_class.to_csv("annotate_batch_01_829_seq_map.csv", sep=";")


In [16]:
# cases_to_annotate_first_batch = cases_to_annotate[cases_to_annotate["study_orthanc_id"].isin(dataset_annotate_class["study_orthanc_id"])]
# cases_to_annotate_first_batch.to_csv("annotate_batch_01_829.csv", sep=";")
